In [21]:
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA,
    HoltWinters,
    CrostonClassic as Croston, 
    HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive
)
from utilsforecast.losses import mse
from utilsforecast.evaluation import evaluate

In [2]:
Y_df = pd.read_parquet('https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet')

In [4]:
Y_df

,unique_id,ds,y
0,H1,1,605.0
1,H1,2,586.0
2,H1,3,586.0
3,H1,4,559.0
4,H1,5,511.0
...,...,...,...
373367,H99,744,24039.0
373368,H99,745,22946.0
373369,H99,746,22217.0
373370,H99,747,21416.0


"This dataset contains 414 unique series with 900 observations on average. For this example and reproducibility’s sake, we will select only 10 unique IDs and keep only the last week. Depending on your processing infrastructure feel free to select more or less series."

In [5]:
uids = Y_df['unique_id'].unique()[:10] # Select 10 ids to make the example faster
Y_df = Y_df.query('unique_id in @uids') # Filter to only those ids
Y_df = Y_df.groupby('unique_id').tail(7 * 24) #Select last 7 days of data to make example faster

In [7]:
StatsForecast.plot(Y_df, engine='plotly')

In [9]:
# Create a list of models and instantiation parameters
models = [
    AutoARIMA(season_length=24),
    HoltWinters(),
    Croston(),
    SeasonalNaive(season_length=24),
    HistoricAverage(),
    DOT(season_length=24)
]

In [10]:
sf = StatsForecast(
    df=Y_df, 
    models=models,
    freq='H', 
    n_jobs=-1,
    fallback_model = SeasonalNaive(season_length=7)
)

StatsForecast achieves its blazing speed using JIT compiling through Numba. The first time you call the statsforecast class, the fit method should take around 5 seconds. The second time -once Numba compiled your settings- it should take less than 0.2s.

In [11]:
forecasts_df = sf.forecast(h=48, level=[90])

In [12]:
sf.plot(Y_df,forecasts_df, engine='plotly')

In [13]:
# Plot to unique_ids and some selected models
sf.plot(Y_df, forecasts_df, models=["HoltWinters","DynamicOptimizedTheta"], unique_ids=["H10", "H105"], level=[90], engine='plotly')

In [14]:
# Explore other models 
sf.plot(Y_df, forecasts_df, models=["AutoARIMA"], unique_ids=["H10", "H105"], level=[90], engine='plotly')

The statsforecast library implements cross-validation as a distributed operation, making the process less time-consuming to perform. If you have big datasets you can also perform Cross Validation in a distributed cluster using Ray, Dask or Spark.

In [17]:
crossvaldation_df = sf.cross_validation(
    df=Y_df,
    h=24,
    step_size=24,
    n_windows=2
)

In [18]:
crossvaldation_df.head()

,ds,cutoff,y,AutoARIMA,HoltWinters,CrostonClassic,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta
unique_id,,,,,,,,,
H1,701,700,619.0,603.925415,847.0,742.668762,691.0,661.674988,612.767517
H1,702,700,565.0,507.591736,820.0,742.668762,618.0,661.674988,536.846252
H1,703,700,532.0,481.281677,790.0,742.668762,563.0,661.674988,497.824280
H1,704,700,495.0,444.410248,784.0,742.668762,529.0,661.674988,464.723236
H1,705,700,481.0,421.168762,752.0,742.668762,504.0,661.674988,440.972351


In [22]:
def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['unique_id', 'ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    # Calculate loss for every unique_id and cutoff.    
    for cutoff in df['cutoff'].unique():
        eval_ = evaluate(df[df['cutoff'] == cutoff], metrics=[metric], models=models)
        evals.append(eval_)
    evals = pd.concat(evals)
    evals = evals.groupby('unique_id').mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [23]:
evaluation_df = evaluate_cross_validation(crossvaldation_df.reset_index(), mse)

In [24]:
evaluation_df.head()

,AutoARIMA,HoltWinters,CrostonClassic,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta,best_model
unique_id,,,,,,,
H1,1979.302368,44888.019531,28038.736328,1422.666748,20927.664062,1296.333984,DynamicOptimizedTheta
H10,458.892700,2812.916504,1483.484131,96.895828,1980.367432,379.621124,SeasonalNaive
H100,8629.948242,121625.375000,91945.140625,12019.000000,78491.187500,21699.648438,AutoARIMA
H101,6818.348633,28453.394531,16183.634766,10944.458008,18208.404297,63698.082031,AutoARIMA
H102,65489.964844,232924.843750,132655.296875,12699.896484,309110.468750,31393.519531,SeasonalNaive


In [25]:
summary_df = evaluation_df.groupby('best_model').size().sort_values().to_frame()
summary_df.reset_index().columns = ["Model", "Nr. of unique_ids"]

In [26]:
seasonal_ids = evaluation_df.query('best_model == "SeasonalNaive"').index
sf.plot(Y_df,forecasts_df, unique_ids=seasonal_ids, models=["SeasonalNaive","DynamicOptimizedTheta"], engine='plotly')

In [27]:
def get_best_model_forecast(forecasts_df, evaluation_df):
    df = forecasts_df.set_index('ds', append=True).stack().to_frame().reset_index(level=2) # Wide to long 
    df.columns = ['model', 'best_model_forecast'] 
    df = df.join(evaluation_df[['best_model']])
    df = df.query('model.str.replace("-lo-90|-hi-90", "", regex=True) == best_model').copy()
    df.loc[:, 'model'] = [model.replace(bm, 'best_model') for model, bm in zip(df['model'], df['best_model'])]
    df = df.drop(columns='best_model').set_index('model', append=True).unstack()
    df.columns = df.columns.droplevel()
    df = df.reset_index(level=1)
    return df

In [28]:
prod_forecasts_df = get_best_model_forecast(forecasts_df, evaluation_df)
prod_forecasts_df.head()

model,ds,best_model,best_model-hi-90,best_model-lo-90
unique_id,,,,
H1,749,592.701843,611.652649,577.677307
H1,750,525.589111,546.621826,505.449738
H1,751,489.251801,512.424133,462.072876
H1,752,456.195038,478.260956,430.554291
H1,753,436.290527,461.815948,411.051239


In [29]:
sf.plot(Y_df, prod_forecasts_df, level=[90], engine='plotly')